In [2]:
import pandas as pd
import numpy as np
df1= pd.read_csv("../hauch/truth_restaurant.csv")
df2= pd.read_csv("../hauch/deception_restaurant.csv")
df3 = pd.read_csv("../hauch/truth_entertainment.csv")
df4= pd.read_csv("../hauch/deception_entertainment.csv")
merged_df = pd.concat([df1, df2, df3, df4], axis=0)
merged_df['average sentence length']  = merged_df['average sentence length'].apply(lambda x: 1 if x == np.inf else x)
merged_df
merged_df.to_csv('all.csv',index=False)

In [3]:
df2.head()

,content,label,words,POS,semantic,word quantity,content word diversity,type-token ratio,two-letter words,average word length,...,Time,Space,Temporal-spatial terms,Prepositions,Numbers,Quantifier,Modifiers,Motion verbs,Cognitive processes,Insight
0,酒店位置明显，很容易就找到了。服务非常周到，菜品味道正宗，菜量大，酒店环境也不错，而且很有特...,1.0,"['酒店', '位置', '明显', '，', '很', '容易', '就', '找到', ...","['n', 'n', 'a', 'wp', 'd', 'a', 'd', 'v', 'u',...","[{'predicate': '明显', 'arguments': [('A0', '酒店位...",39,31,0.794872,0,1.512821,...,0.051282,0.000000,0.051282,0.000000,0.000000,0.000000,0.0,0.025641,0.256410,0.051282
1,朋友介绍的一家店，总体感觉蛮不错的，装修得很雅致，在这种环境下用餐心情很愉快～特别喜欢吃他家...,1.0,"['朋友', '介绍', '的', '一', '家', '店', '，', '总体', '感...","['n', 'v', 'u', 'm', 'q', 'n', 'wp', 'n', 'n',...","[{'predicate': '介绍', 'arguments': [('A0', '朋友'...",73,57,0.780822,2,1.465753,...,0.000000,0.027397,0.027397,0.013699,0.013699,0.013699,0.0,0.013513,0.108108,0.040541
2,对这家餐厅总体很满意，菜量大，味道好，客家酿豆腐好吃，豆腐嫩，红烧桂花鱼味道好，满意。它的环...,1.0,"['对', '这家', '餐厅', '总体', '很', '满意', '，', '菜量', ...","['p', 'r', 'n', 'n', 'd', 'v', 'wp', 'n', 'a',...","[{'predicate': '满意', 'arguments': [('A1', '对这家...",44,31,0.704545,0,1.545455,...,0.000000,0.000000,0.000000,0.022727,0.000000,0.000000,0.0,0.000000,0.162791,0.069767
3,好久没来广东！回来到这边和爹娘一起来吃。总体环境非常好，很享受，环境也十分有特色。是比较传统...,1.0,"['好久', '没', '来', '广东', '！', '回来', '到', '这边', '...","['m', 'd', 'v', 'ns', 'wp', 'v', 'v', 'r', 'p'...","[{'predicate': '来', 'arguments': [('ARGM-ADV',...",63,47,0.746032,0,1.428571,...,0.015873,0.015873,0.031746,0.015873,0.031746,0.000000,0.0,0.100000,0.266667,0.033333
4,环境很好，服务也很到位，菜量很多又好吃，味道又正宗，酒店所在位置很明显，房间阳光很充足，也很...,1.0,"['环境', '很', '好', '，', '服务', '也', '很', '到位', '，...","['n', 'd', 'a', 'wp', 'v', 'd', 'd', 'v', 'wp'...","[{'predicate': '好', 'arguments': [('A0', '环境')...",46,28,0.608696,0,1.347826,...,0.000000,0.021739,0.021739,0.000000,0.000000,0.000000,0.0,0.000000,0.209302,0.000000


In [4]:
# from sklearn.svm import SVC
# from sklearn.model_selection import KFold,GridSearchCV,cross_validate
# from sklearn.preprocessing import StandardScaler

In [5]:
# 数据标准化 
from sklearn.preprocessing import StandardScaler
X = merged_df.iloc[:, 5:]
y = merged_df['label']
scaler = StandardScaler()
scaler.fit(X)
X_std = scaler.transform(X)

In [6]:
# 交叉验证
from sklearn.model_selection import KFold
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

In [10]:
# SVM
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate, cross_val_score
from bayes_opt import BayesianOptimization

svm = SVC()

def svm_cv(C):
    scores = cross_val_score(
        SVC(kernel='linear',
            C=C
        ),
        X_std, y, scoring='accuracy', cv=kfold 
    ).mean()
    return scores

svm_bo = BayesianOptimization(
    svm_cv,
    {'C': (1e-04, 1e04),}
)

svm_bo.maximize()
print(svm_bo.res)


|   iter    |  target   |     C     |
-------------------------------------
| 1         | 0.8834    | 3.124e+03 |
| 2         | 0.8834    | 1.654e+03 |
| 3         | 0.8844    | 678.1     |
| 4         | 0.8834    | 1.467e+03 |
| 5         | 0.8844    | 1.167e+03 |
| 6         | 0.8844    | 679.3     |
| 7         | 0.8834    | 153.1     |
| 8         | 0.8834    | 968.3     |
| 9         | 0.8854    | 1.265e+03 |
| 10        | 0.8844    | 1.322e+03 |
| 11        | 0.8863    | 1.23e+03  |
| 12        | 0.8854    | 1.212e+03 |
| 13        | 0.8863    | 1.242e+03 |
| 14        | 0.8824    | 7.764e+03 |
| 15        | 0.8824    | 519.4     |
| 16        | 0.8824    | 5.538e+03 |
| 17        | 0.8824    | 4.486e+03 |
| 18        | 0.8834    | 797.5     |
| 19        | 0.8815    | 6.433e+03 |
| 20        | 0.8834    | 1.085e+03 |
| 21        | 0.8815    | 8.697e+03 |
| 22        | 0.8805    | 9.543e+03 |
| 23        | 0.8834    | 2.39e+03  |
| 24        | 0.8824    | 3.778e+03 |
| 25        

In [11]:
svm = SVC(kernel='linear', C=1230)
scores = cross_validate(svm, X_std, y, cv=kfold, scoring=('accuracy','precision','recall','roc_auc'),return_train_score=True)
print('Accuracy:', scores['test_accuracy'].mean())
print('Precision:', scores['test_precision'].mean())
print('Recall:', scores['test_recall'].mean())
print('Auc:', scores['test_roc_auc'].mean())



Accuracy: 0.8853512278697886
Precision: 0.8627890330012947
Recall: 0.9120903168942522
Auc: 0.9330431832091168


In [20]:
from sklearn.model_selection import GridSearchCV
model = SVC(probability=True)
params = [
{'kernel':['rbf'],'C':[1,10,100,1000], 
 'gamma':[1,0.1, 0.01, 0.001]}]
model = GridSearchCV(estimator=model, param_grid=params, cv=10)	 
model.fit(X_std, y)

# 网格搜索训练后的副产品
print("模型的最优参数：",model.best_params_)
print("最优模型分数：",model.best_score_)
print("最优模型对象：",model.best_estimator_)


模型的最优参数： {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
最优模型分数： 0.8727108319055776
最优模型对象： SVC(C=100, gamma=0.001, probability=True)


In [30]:
from xgboost import XGBClassifier


xgb = XGBClassifier()

def xgb_cv(eta,gamma,alpha,la,max_depth,min_child_weight,ubsample,colsample_bytree,n_estimators):
    scores = cross_val_score(
        xgb(kernel='linear',
            eta=eta,
            gamma=gamma,
            alpha=alpha,
            la='lambda',
            max_depth=max_depth,
            min_child_weight=min_child_weight,
            ubsample=ubsample,
            colsample_bytree=colsample_bytree,
            n_estimators=n_estimators     
        ),
        X_std, y, scoring='roc_auc', cv=kfold 
    ).mean()
    return scores



xgb_bo = BayesianOptimization(
    xgb_cv,
    {'eta':(0.01,0.1),
    'gamma':(0, 1),
    'alpha':(0,1),
    'la':(0,1),
    'max_depth':(1,50),
    'min_child_weight':(1,10),
    'subsample':(0,1),
    'colsample_bytree':(0,1),
    'n_estimators':(100,500)}
)

xgb_bo.maximize()
print(svm_bo.res)

# xgb = XGBClassifier()
# params = {
#     'eta':[0.01, 0.015, 0.025, 0.05, 0.1],
#     'gamma':[0, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 1] ,
#     'alpha':[0, 0.01,0.05, 0.1, 1],
#     'lambda':[0, 0.1, 0.5, 1],
#     'max_depth':[3, 5, 6, 7, 9, 12, 15, 17, 25],
#     'min_child_weight':[1, 3, 5, 7],
#     'subsample':[0.6, 0.7, 0.8, 0.9, 1],
#     'colsample_bytree':[0.6, 0.7, 0.8, 0.9, 1],
#     'n_estimators':[100, 200, 300]
# }

# model = GridSearchCV(estimator=xgb, param_grid=params, cv=5)	 
# model.fit(X, Y)

# # 网格搜索训练后的副产品
# print("模型的最优参数：",model.best_params_)
# print("最优模型分数：",model.best_score_)
# print("最优模型对象：",model.best_estimator_)


|   iter    |  target   |   alpha   | colsam... |    eta    |   gamma   |    la     | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------


TypeError: xgb_cv() got an unexpected keyword argument 'subsample'

In [1]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores_xgb = cross_validate(xgb , X_std, y, cv=kfold, scoring=('accuracy','precision','recall','roc_auc'),return_train_score=True)
print('Accuracy:', scores_xgb['test_accuracy'].mean())
print('Precision:', scores_xgb['test_precision'].mean())
print('Recall:', scores_xgb['test_recall'].mean())
print('Auc:', scores_xgb['test_roc_auc'].mean())


NameError: name 'KFold' is not defined